# Preprocessing of the news articles

## Reuters

In [2]:
import pandas as pd
import string
import re
from string import digits
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
import gensim
import matplotlib.pyplot as plt
import urllib.parse

In [3]:
file1 = pd.read_csv("../news_data/reuters2019-10-26.csv")
file2 = pd.read_csv("../news_data/reuters2019-10-27.csv")
mergedReutersFiles = pd.merge( file1, file2, on=['headline','date','content'],how = 'outer')
mergedReutersFiles.dropna( inplace = True )
null_columns = mergedReutersFiles.columns[mergedReutersFiles.isnull().any()]
print(mergedReutersFiles[mergedReutersFiles.isnull().any(axis=1)][null_columns])
mergedReutersFiles

Empty DataFrame
Columns: []
Index: []


,headline,date,content
0,JS Global pulls Hong Kong listing amid trade g...,"October 26, 2019 / 9:39 AM / Updated 2 minute...",HONG KONG (Reuters) - Household appliances mak...
1,Tech consultations on parts of U.S. trade deal...,"October 26, 2019 / 4:02 AM / Updated 2 hours ago",SHANGHAI (Reuters) - Chinese Vice Premier Liu ...
2,Lagarde will take ECB out of ivory tower: De G...,"October 26, 2019 / 8:35 AM / Updated 2 hours ago",FRANKFURT (Reuters) - Christine Lagarde will e...
3,"U.S., China say they are 'close to finalizing'...","October 25, 2019 / 5:08 AM / Updated 6 hours ago",WASHINGTON (Reuters) - U.S. and Chinese offici...
4,"S&P 500 nears record high on trade progress, u...","October 25, 2019 / 11:53 AM / Updated 8 hours...",NEW YORK (Reuters) - U.S. stocks rose on Frida...
5,Microsoft beats Amazon for Pentagon's $10 bill...,"October 25, 2019 / 11:23 PM / Updated 8 hours...",WASHINGTON (Reuters) - Microsoft Corp. (MSFT.O...
6,UAW turns to Ford after GM workers approve new...,"October 24, 2019 / 9:00 PM / Updated 9 hours ago",DETROIT/WASHINGTON (Reuters) - The United Auto...
7,General Motors to restart Mexican plants after...,"October 26, 2019 / 1:04 AM / Updated 9 hours ago",MEXICO CITY (Reuters) - General Motors Co (GM....
8,Johnson & Johnson CEO testified Baby Powder wa...,"October 22, 2019 / 8:03 PM / Updated 9 hours ago",LOS ANGELES/NEW YORK (Reuters) - Facing off ag...
9,Timeline: UAW members ratify new labor deal wi...,"October 25, 2019 / 10:19 PM / Updated 13 hour...",(Reuters) - The United Auto Workers union said...


In [4]:
 def preprocessing(x):
#     x = x.replace('—', ' ')
    x = x.replace('(Reuters)', ' ')
    x = re.sub('â€™', "'", x)
    x = x.replace ( '[^\x00-\x7f]', 's' )
    
    return x

def processing( dates ):
    dates = dates.split( '/' )
    final_date = dates[0] + '/' + dates[1]
    
    return final_date
mergedReutersFiles['content'] = mergedReutersFiles['content'].apply(preprocessing)
mergedReutersFiles['date'] = mergedReutersFiles['date'].apply(processing)
mergedReutersFiles.index = mergedReutersFiles.index + 1
cols = [ 'content','headline','date' ]
mergedReutersFiles = mergedReutersFiles[cols]
mergedReutersFiles.rename( columns = { 'date' : 'time' }, inplace = True )
mergedReutersFiles

,content,headline,time
1,HONG KONG - Household appliances maker JS Gl...,JS Global pulls Hong Kong listing amid trade g...,"October 26, 2019 / 9:39 AM"
2,SHANGHAI - Chinese Vice Premier Liu He had a...,Tech consultations on parts of U.S. trade deal...,"October 26, 2019 / 4:02 AM"
3,FRANKFURT - Christine Lagarde will ensure Eu...,Lagarde will take ECB out of ivory tower: De G...,"October 26, 2019 / 8:35 AM"
4,WASHINGTON - U.S. and Chinese officials are ...,"U.S., China say they are 'close to finalizing'...","October 25, 2019 / 5:08 AM"
5,NEW YORK - U.S. stocks rose on Friday after ...,"S&P 500 nears record high on trade progress, u...","October 25, 2019 / 11:53 AM"
6,WASHINGTON - Microsoft Corp. (MSFT.O) has wo...,Microsoft beats Amazon for Pentagon's $10 bill...,"October 25, 2019 / 11:23 PM"
7,DETROIT/WASHINGTON - The United Auto Workers...,UAW turns to Ford after GM workers approve new...,"October 24, 2019 / 9:00 PM"
8,MEXICO CITY - General Motors Co (GM.N) will ...,General Motors to restart Mexican plants after...,"October 26, 2019 / 1:04 AM"
9,LOS ANGELES/NEW YORK - Facing off against a ...,Johnson & Johnson CEO testified Baby Powder wa...,"October 22, 2019 / 8:03 PM"
10,- The United Auto Workers union said on Frid...,Timeline: UAW members ratify new labor deal wi...,"October 25, 2019 / 10:19 PM"


In [5]:
mergedReutersFiles.to_csv('../news_data/merged_cleaned_Reuters.csv',encoding='utf-8-sig')


## The Guardians

In [8]:
file1 = pd.read_csv("../news_data/the_guardian2019-11-09_first_part.csv")
file2 = pd.read_csv("../news_data/the_guardian2019-11-09_second_part.csv")
mergedGuardians = pd.merge( file1, file2 ,on=['headline','standfirst', 'author', 'date_time','content', 'url'],how = 'outer')
mergedGuardians.dropna( inplace = True )

In [9]:
mergedGuardians

,headline,standfirst,author,date_time,content,url
1,Brexit 'do or die': what are Boris Johnson's o...,The prime minister’s call for a December elect...,Heather Stewart,2019-10-24T21:31:14+0100,Boris Johnson set out a new ultimatum on Thurs...,https://www.theguardian.com/politics/2019/oct/...
2,Pollutionwatch: European air-quality map expos...,From diesel-stricken Paris to coal-reliant Pol...,Gary Fuller,2019-10-24T21:30:06+0100,The European Environment Agency has shed fresh...,https://www.theguardian.com/environment/2019/o...
3,EU set to put Brexit delay on hold after Johns...,PM’s threat to pull deal if Corbyn rejects gen...,Daniel Boffey,2019-10-24T21:23:36+0100,The EU’s plan to offer the UK an extension unt...,https://www.theguardian.com/politics/2019/oct/...
4,White House to federal agencies: cancel New Yo...,Government staffers instructed not to renew ne...,Joan E Greve,2019-10-24T20:42:26+0100,The White House has instructed federal agencie...,https://www.theguardian.com/us-news/2019/oct/2...
5,The Guardian view on the migrant tragedy in Es...,Editorial: An international response is requir...,Editorial,2019-10-24T19:14:12+0100,The known facts about how 39 people died in a ...,https://www.theguardian.com/uk-news/2019/oct/2...
6,AfD gains expected in battle for political hea...,Party whose leader is labelled a Nazi is on pa...,Kate Connolly,2019-10-24T18:59:19+0100,Right-wing populists are expected to make stro...,https://www.theguardian.com/world/2019/oct/24/...
7,Draghi bows out at ECB with warning on eurozon...,"Mario Draghi blames slowing global economy, Br...",Phillip Inman,2019-10-24T18:59:17+0100,"Mario Draghi, the outgoing president of the Eu...",https://www.theguardian.com/business/2019/oct/...
8,China-UK people trafficking often driven by de...,Many victims travel to UK to pay off gambling ...,Diane Taylor,2019-10-24T18:54:07+0100,Chinese migrants hoping to reach the UK can ty...,https://www.theguardian.com/uk-news/2019/oct/2...
9,At last Franco is properly buried. This victor...,Now the last great symbol of the dictatorship ...,Irene Lozano,2019-10-24T18:44:41+0100,"In Spain, the process to exhume the remains of...",https://www.theguardian.com/commentisfree/2019...
10,TikTok app poses potential national security r...,"Chuck Schumer and Tom Cotton urged inquiry, no...",Adam Gabbatt,2019-10-24T18:18:44+0100,"Chuck Schumer, the most senior Democrat in the...",https://www.theguardian.com/technology/2019/oc...


In [10]:
def processing( dates ):
    dates = dates.replace('T', '/ ')
    dates = dates.replace('+0100','')
    return dates

mergedGuardians['date_time'] = mergedGuardians['date_time'].apply(processing)


#mergedGuardians.drop('url',axis = 1)
cols = ['author', 'content','headline', 'standfirst', 'date_time']
mergedGuardians = mergedGuardians[cols]
mergedGuardians.rename( columns = { 'date_time' : 'time' }, inplace = True )
mergedGuardians = mergedGuardians

/Users/yuankanglee/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [11]:
mergedGuardians


,author,content,headline,standfirst,time
1,Heather Stewart,Boris Johnson set out a new ultimatum on Thurs...,Brexit 'do or die': what are Boris Johnson's o...,The prime minister’s call for a December elect...,2019-10-24/ 21:31:14
2,Gary Fuller,The European Environment Agency has shed fresh...,Pollutionwatch: European air-quality map expos...,From diesel-stricken Paris to coal-reliant Pol...,2019-10-24/ 21:30:06
3,Daniel Boffey,The EU’s plan to offer the UK an extension unt...,EU set to put Brexit delay on hold after Johns...,PM’s threat to pull deal if Corbyn rejects gen...,2019-10-24/ 21:23:36
4,Joan E Greve,The White House has instructed federal agencie...,White House to federal agencies: cancel New Yo...,Government staffers instructed not to renew ne...,2019-10-24/ 20:42:26
5,Editorial,The known facts about how 39 people died in a ...,The Guardian view on the migrant tragedy in Es...,Editorial: An international response is requir...,2019-10-24/ 19:14:12
6,Kate Connolly,Right-wing populists are expected to make stro...,AfD gains expected in battle for political hea...,Party whose leader is labelled a Nazi is on pa...,2019-10-24/ 18:59:19
7,Phillip Inman,"Mario Draghi, the outgoing president of the Eu...",Draghi bows out at ECB with warning on eurozon...,"Mario Draghi blames slowing global economy, Br...",2019-10-24/ 18:59:17
8,Diane Taylor,Chinese migrants hoping to reach the UK can ty...,China-UK people trafficking often driven by de...,Many victims travel to UK to pay off gambling ...,2019-10-24/ 18:54:07
9,Irene Lozano,"In Spain, the process to exhume the remains of...",At last Franco is properly buried. This victor...,Now the last great symbol of the dictatorship ...,2019-10-24/ 18:44:41
10,Adam Gabbatt,"Chuck Schumer, the most senior Democrat in the...",TikTok app poses potential national security r...,"Chuck Schumer and Tom Cotton urged inquiry, no...",2019-10-24/ 18:18:44


In [12]:
mergedGuardians.to_csv('../news_data/merged_cleaned_TheGuardians.csv', encoding='utf-8-sig')